**Decision Tree Titanic**

*To upload .csv file from my local drive:*

In [2]:
from google.colab import files
uploaded = files.upload()

Saving Titanic_Data.csv to Titanic_Data.csv


To import it into a **dataframe** (make sure the filename matches the name of the uploaded file).

In [92]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model

import io
df = pd.read_csv(io.BytesIO(uploaded['Titanic_Data.csv']))
# print(df)

pd.set_option('display.max_columns', None)
df.head(10)

# Only useable Columns are: Survived;  Pclass;  Sex;  Age;	Fare;

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


**Clear the Data frame:**

*My only useable Columns are: Survived;  Pclass;  Sex;  Age;	Fare;. So, drop all columns except above 5 columns.*

In [93]:
Clear_Data = df.drop(['PassengerId', 'Name', 'SibSp',	'Parch', 'Ticket', 'Cabin',	'Embarked'],axis='columns')

Clear_Data.head(10)

,Survived,Pclass,Sex,Age,Fare
0,0,3,male,22.0,7.2500
1,1,1,female,38.0,71.2833
2,1,3,female,26.0,7.9250
3,1,1,female,35.0,53.1000
4,0,3,male,35.0,8.0500
5,0,3,male,NaN,8.4583
6,0,1,male,54.0,51.8625
7,0,3,male,2.0,21.0750
8,1,3,female,27.0,11.1333
9,1,2,female,14.0,30.0708


**Prepare inputs Data:**

In [94]:
# Clear_Data.drop(['Survived'])
inputs = Clear_Data.drop('Survived',axis='columns')
inputs.head(10)

,Pclass,Sex,Age,Fare
0,3,male,22.0,7.2500
1,1,female,38.0,71.2833
2,3,female,26.0,7.9250
3,1,female,35.0,53.1000
4,3,male,35.0,8.0500
5,3,male,NaN,8.4583
6,1,male,54.0,51.8625
7,3,male,2.0,21.0750
8,3,female,27.0,11.1333
9,2,female,14.0,30.0708


Clean *Age* column. Because *Age* column contain NaN values. And show ALL NaN valu of Age column:

In [95]:
# df1 = inputs[inputs.isna().any(axis=1)]

NaN_Age = inputs.loc[inputs['Age'].isnull()]
NaN_Age.head(10)

,Pclass,Sex,Age,Fare
5,3,male,NaN,8.4583
17,2,male,NaN,13.0000
19,3,female,NaN,7.2250
26,3,male,NaN,7.2250
28,3,female,NaN,7.8792
29,3,male,NaN,7.8958
31,1,female,NaN,146.5208
32,3,female,NaN,7.7500
36,3,male,NaN,7.2292
42,3,male,NaN,7.8958


Replace NaN by *mean()* value of *Age*:

In [96]:
inputs.Age = inputs.Age.fillna(inputs.Age.mean())

Now **inputs** data is clean:

In [97]:
inputs.head(10)

,Pclass,Sex,Age,Fare
0,3,male,22.000000,7.2500
1,1,female,38.000000,71.2833
2,3,female,26.000000,7.9250
3,1,female,35.000000,53.1000
4,3,male,35.000000,8.0500
5,3,male,29.699118,8.4583
6,1,male,54.000000,51.8625
7,3,male,2.000000,21.0750
8,3,female,27.000000,11.1333
9,2,female,14.000000,30.0708


**Now replace Sex male=1; female=2;**

In [98]:
inputs.Sex = inputs.Sex.map({'male': 1, 'female': 2})
inputs.head(10)

,Pclass,Sex,Age,Fare
0,3,1,22.000000,7.2500
1,1,2,38.000000,71.2833
2,3,2,26.000000,7.9250
3,1,2,35.000000,53.1000
4,3,1,35.000000,8.0500
5,3,1,29.699118,8.4583
6,1,1,54.000000,51.8625
7,3,1,2.000000,21.0750
8,3,2,27.000000,11.1333
9,2,2,14.000000,30.0708


**Prepare target Data:**

In [99]:
target = Clear_Data.Survived
target.head(10)

0    0
1    1
2    1
3    1
4    0
5    0
6    0
7    0
8    1
9    1
Name: Survived, dtype: int64

**Splite data for training and testing purpose (20%) :**

In [100]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(inputs,target,test_size=0.2)

*Size of train & testing Data:*

In [101]:
print(f"Length of X_test Data: {len(X_test)}")
print(f"Length of X_train Data: {len(X_train)}")

Length of X_test Data: 179
Length of X_train Data: 712



**Create Decision Tree Classifier Model:**

In [102]:
from sklearn import tree
model = tree.DecisionTreeClassifier()

*Train the Model through Splite Data:*

In [103]:
model.fit(X_train,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

**TEST-01 / (Prediction-01):**

In [105]:
model.predict([['1',	'2', '38.000000', '71.2833']])

array([1])

**TEST-02 (Prediction-02):**

In [106]:
model.predict([['3',	'1',	'35.000000',	'8.0500']])

array([0])

**Accuracy of the model:**

In [109]:
model.score(X_test,y_test)

0.8268156424581006

In [110]:
print("Made by AMARTA KUNDU. IT, JISCE.")

Made by AMARTA KUNDU. IT, JISCE.
